In [131]:
"""
Goal: implement differential expression analysis
for two Leukemia conditions usig the Z-test.

Notes:

- The data for this analysis was downloaded from
    http://portals.broadinstitute.org/cgi-bin/cancer/publications/pub_paper.cgi?mode=view&paper_id=43
- This code is an adaptation for pandas of the code provided at
    http://dept.stat.lsa.umich.edu/~kshedden/Python-Workshop/gene_expression_comparison.html
    which uses data in "SOFT" format -- https://www.ncbi.nlm.nih.gov/sites/GDSbrowser?acc=GDS1615


@author: Andrei Sura
@see https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3134237/ - Gene Set Enrichment Analysis Made Simple
@see http://www.gettinggeneticsdone.com/2012/03/pathway-analysis-for-high-throughput.html

#########################################################
# Expression profiling in early onset colorectal cancer
@see http://clincancerres.aacrjournals.org/content/13/4/1107.full-text.pdf
@see https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE4107

@see http://isites.harvard.edu/fs/docs/icb.topic1517419.files/golub_analysis.R
@see http://svitsrv25.epfl.ch/R-doc/library/multtest/doc/golub.R

@see https://www.biostars.org/p/16137/
@see http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0012336
@see http://bioinformatics.mdanderson.org/MicroarrayCourse/Lectures09/ma10b.pdf
@see http://sebastianraschka.com/Articles/2014_pca_step_by_step.html

@see https://www.biostars.org/p/42219/ - pathway analysis
"""

import pandas as pd
import numpy as np
from scipy.stats.distributions import norm
# np.seterr(invalid='ignore')

IN_FILE = '/Users/asura/git/gene_expression_two_group_comparison/_golub_assignment/data_set_ALL_AML_train.csv'
OUT_FILE = '/Users/asura/git/gene_expression_two_group_comparison/_golub_assignment/out.csv'

# RANGE_ALL = ["call.{}".format(x) for x in range(0, 27)]
# RANGE_AML = ["call.{}".format(x) for x in range(27, 38)]
RANGE_ALL = ["call_{}".format(x) for x in range(1, 28)]
RANGE_AML = ["call_{}".format(x) for x in range(28, 39)]


"""
38 samples divided in two groups:
- ALL: 27 (1 to 27)  
- AML: 11 (28 to 38)

An accession number in bioinformatics is a unique identifier given to a DNA
or protein sequence record to allow for tracking of different versions of
that sequence record and the associated sequence over time in a single data
repository. Because of its relative stability, accession numbers can be 
utilized as foreign keys for referring to a sequence object, but not necessarily
to a unique sequence. All sequence information repositories implement the
concept of "accession number" but might do so with subtle variations.

"""
df = pd.read_csv(IN_FILE, sep='\t', skipinitialspace=False, nrows=None)
df['descr'] = df.index

# df.reset_index(inplace=True, drop=False)
# df.reindex()
# df.set_index(range(len(df)))

# fix weird error with reading data
del df['call_33']
df.rename(columns={"call": "call.0", "Gene_Accession_Number": "call_33"}, inplace=True)
# sorted(df)
df.head()

,Gene_Description,call_33,data_1,call_1,data_2,call_2,data_3,call_3,data_4,call_4,...,data_29,call_29,data_30,call_30,data_31,call_31,data_32,call_32,data_33,descr
AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,-106,...,A,-318,A,-32,A,-124,A,-135,A,AFFX-BioB-5_at (endogenous control)
AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,-125,...,A,-192,A,-49,A,-79,A,-186,A,AFFX-BioB-M_at (endogenous control)
AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,-58,A,-1,A,-307,A,265,A,-76,...,A,-95,A,49,A,-37,A,-70,A,AFFX-BioB-3_at (endogenous control)
AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,88,A,283,A,309,A,12,A,168,...,A,312,A,230,P,330,A,337,A,AFFX-BioC-5_at (endogenous control)
AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-295,A,-264,A,-376,A,-419,A,-230,...,A,-139,A,-367,A,-188,A,-407,A,AFFX-BioC-3_at (endogenous control)


In [132]:
columns = ["descr"]
columns_data = []
columns_data.extend(RANGE_ALL)
columns_data.extend(RANGE_AML)

columns.extend(RANGE_ALL)
columns.extend(RANGE_AML)
df = df[columns]
# df.describe()
# sorted(df)
# df.head(10)
df_all = df[RANGE_ALL]
df_aml = df[RANGE_AML]
df_aml.head()



,call_28,call_29,call_30,call_31,call_32,call_33,call_34,call_35,call_36,call_37,call_38
AFFX-BioB-5_at (endogenous control),15,-318,-32,-124,-135,-214,7,-213,-25,-72,-4
AFFX-BioB-M_at (endogenous control),-114,-192,-49,-79,-186,-153,-100,-252,-20,-139,-116
AFFX-BioB-3_at (endogenous control),2,-95,49,-37,-70,-58,-57,136,124,-1,-125
AFFX-BioC-5_at (endogenous control),193,312,230,330,337,88,132,318,325,392,241
AFFX-BioC-3_at (endogenous control),-51,-139,-367,-188,-407,-295,-377,-209,-396,-324,-191


In [133]:


# df_all = df_all.applymap(lambda x: np.log(x) / np.log(2))
# df_aml = df_aml.applymap(lambda x: np.log(x) / np.log(2))

mean_all = df_all.mean(axis=1)  # means for 7129 genes
mean_aml = df_aml.mean(axis=1)   
var_all = df_all.var(axis=1)  # variance for 7129 genes
var_aml = df_aml.var(axis=1)

num_all = len(RANGE_ALL)  # number of samples of each condition
num_aml = len(RANGE_AML)

Z = (mean_all - mean_aml) / np.sqrt(var_all/num_all + var_aml/num_aml)
# print("means ALL: {}\n {}".format(len(mean_all), mean_all))
# print("means AML: {}\n {}".format(len(mean_aml), mean_aml))

print("Z mean: {}".format(Z.mean()))
print("Z std: {}".format(Z.std()))

"""
Z mean: 0.14004765578280895
Z std: 1.6058829875390022
Z means: 0.22008696871931546, 0.13073362322906437, 0.8692663767709357

Since the standard deviation is greater than 1, there appear to be multiple
genes for which the mean expression levels in the ALL and AML samples differ. 
Further, since the mean Z-score is positive, it appears that the dominant pattern
is for genes to be expressed at a higher level in the ALL compared to the AML samples.
"""

mean_z_mod = np.mean(np.abs(Z) > 2)
mean_z1 = np.mean(Z > 2)
mean_z2 = np.mean(Z < 2)

print("Z means: {}, {}, {}".format(mean_z_mod, mean_z1, mean_z2))



Z mean: 0.140047655782809
Z std: 1.6058829875390022
Z means: 0.22008696871931546, 0.13073362322906437, 0.8692663767709357


In [134]:
"""
Genes with low expression level are harder to measure accurately, 
thus we expect that fewer of these genes will meet a given statistical
threshold for differential expression. Similarly, genes with low variance
are potentially less likely to be associated with biological differences.
We can assess these trends in our data. First, we will determine which
genes are in the lower and upper halves of all genes based on either 
mean expression or expression variation.
"""
SD = df[columns_data].std()
index_stdev_low = np.flatnonzero(SD < np.median(SD))
index_stdev_high = np.flatnonzero(SD > np.median(SD))

MEAN = df[columns_data].mean()
index_mean_low = np.flatnonzero(MEAN < np.median(MEAN))
index_mean_high = np.flatnonzero(MEAN > np.median(MEAN))

"""
Now we can look at the proportion of genes within each of these
strata that have Z-score magnitude greater than two.
"""
mean_low = np.mean(np.abs(Z[index_stdev_low]) > 2)
mean_high = np.mean(np.abs(Z[index_stdev_high]) > 2)

print("Check the mean of values with low Z magnitude: {}".format(mean_low))
print("Check the mean of values with high Z magnitude: {}".format(mean_high))



Check the mean of values with low Z magnitude: 0.21052631578947367
Check the mean of values with high Z magnitude: 0.21052631578947367


In [135]:
## The Z-score threshold under a Bonferroni correction
zst = -norm.ppf(0.025/Z.shape[0])
indexes = np.flatnonzero(np.abs(Z) > zst)

with open(OUT_FILE, "w") as fout:
    for i in indexes:
        # print("Found gene: {} - {}".format(i, df["Gene Description"][i]))
        print("{} - {}".format(i, df["descr"][i]))
        fout.write("{},{}\n".format(Z[i], df["descr"][i]))

148 - Clone 22 mRNA, alternative splice variant alpha-1
531 - HMG1 High-mobility group (nonhistone chromosomal) protein 1
1134 - ASPARTYL-TRNA SYNTHETASE
1143 - SPTAN1 Spectrin, alpha, non-erythrocytic 1 (alpha-fodrin)
1206 - Protein tyrosine kinase related mRNA sequence
1244 - GBE1 Glucan (1,4-alpha-), branching enzyme 1 (glycogen branching enzyme, Andersen disease, glycogen storage disease type IV)
1629 - Inducible protein mRNA
1703 - ADA Adenosine deaminase
1810 - MDU1 Antigen identified by monoclonal antibodies 4F2, TRA1.10, TROP4, and T43
1923 - PRKAR2B Protein kinase, cAMP-dependent, regulatory, type II, beta
2019 - FAH Fumarylacetoacetate
2300 - ACTN2 Actinin alpha 2 
2440 - HKR-T1
2625 - MSH2 DNA repair protein MSH2
2758 - Thrombospondin-p50 gene extracted from Human thrombospondin-1 gene, partial cds
3069 - Tax1-binding protein TXBP181 mRNA
3311 - Protein kinase ATR mRNA
3319 - Leukotriene C4 synthase (LTC4S) gene
3846 - GB DEF = Homeodomain protein HoxA9 mRNA
4229 - CTPS CTP 